*Copyright statement:
This material, no matter whether in printed or electronic form, may be used for personal and non-commercial educational use only. Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.*

# Polynomial Regression

The goal of polynomial regression (and also linear regression) is to find the function which best models the relationship between an independent variable and a dependent variable as an degree-$d$ polynomial. With polynomial regression, this means finding the coefficients that provide the best fit to a non-linear relationship in the data, which a straight line (linear regression) cannot always do precisely.

Imagine that $y$ is our true underlying polynomial, which we often do not know. However, we might know data points $x$ sampled around our true polynomial, and we want to find the polynomial $\hat y$ that approximates $y$ given our data. In polynomial regression, we can model this relationship as:

$$y \approx \hat y = \theta_0 x_p^0 + \theta_1 x_p^1 + \theta_2 x_p^2 + \theta_3 x_p^3 + ... + \theta_d x_p^d$$

where the thetas $\theta_0, \theta_1, ..., \theta_d$ are the parameters/weights we are trying to find and the $x_p$ are polynomial feature vectors $\in \mathbb{R}^{n}$ where $n$ is the number of sampled datapoints. We can also use matrix notation and summarize our parameters and our polynomial features as matrix-vector multiplication:

$$y \approx \hat y = X_{poly} \mathbf{\theta}$$

Where $X_{poly} \in \mathbb{R}^{n \times d+1}$ is the polynomial feature matrix and $\mathbf{\theta}$ are the weights. Imagine you are trying to fit a degree-3 polynomial (i.e. $\theta_0 x^0 + \theta_1 x^1 + \theta_2 x^2 + \theta_3 x^3$) to $n=3$ datapoints $x = [2, 3, 5]$. The polynomial feature matrix then expands the input vector $x$ into multiple features like so:

$$X_{poly} = \begin{bmatrix}
2^0 & 2^1 & 2^2 & 2^3 \\
3^0 & 3^1 & 3^2 & 3^3 \\
5^0 & 5^1 & 5^2 & 5^3
\end{bmatrix}$$

hence a $(n \times d+1)$ aka a $(3 \times 4)$ matrix, since we are factoring in the x's to the power of 0 as well. The weight vector $\theta$ is of dimensionality $(d + 1 \times 1)$ after making it a column vector, hence we get the model's output dimensionality of $(3 \times 4) * (4 \times 1) = (3 \times 1)$, which makes sense considering we have the same input dimensions.

Throughout this notebook, we will:

*    Generate synthetic, noisy data we are trying to fit a function to
*    Create a polynomial feature matrix to make a linear model fit non-linear data
*    Finding the optimal weights through least squares
*    Evaluating the generalization error and empirical risk through the Mean Squared Error
* Intercatively visualize the impact of different settings

We provide some intermediate checks to make sure your outputs are correct so you can move on to the next task. First, we import all necessary packages:


In [ ]:
!pip install ipympl

In [2]:
# necessary imports
import torch

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from ipywidgets import interact
%matplotlib widget

Next, we set a seed for deterministic data generation:

In [18]:
# seed for reproducibility
seed = 42
torch.manual_seed(seed)

### Data generation

Now let us generate our data. We define the sine function as our true underlying function and generate some datapoints around it. To make it look more like real data you could have gained from an experiment, we add some Gaussian noise as well. In general, training on realistic/noisy data also makes sure that our model becomes more robust and can generalize even if outliers are present, as it helps simulate the unpredictability of the real world.

We first create a function `true_polynomial` which models our sine function, aka our true polynomial $y$. Here $x \in \mathbb{R}$ is expected to be between 0 and 1, so we obtain a sine wave that completes a full period ($2 \pi$) between 0 and 1.

In [4]:
def true_polynomial(x):
    return torch.sin(2 * torch.pi * x)

We also provide the functionality for generating datapoints $x$ through `generate_datapoints`, which takes in `n_samples` the number of datapoints we want to create around our true polynomial, and `noise` the degree of noise in our data, aka the spread of our data around the true function.

In [24]:
def generate_datapoints(n_samples, noise):
    torch.manual_seed(seed)
    X = torch.rand(n_samples, 1) # n_samples in [0,1)
    # obtain noisy observations around the true polynomial
    noisy_y = noise * torch.randn(n_samples) # generate gaussian noise 
    y = true_polynomial(X.squeeze()) + noisy_y # y = y + noise
    return X, y

### Task 1: Implement polynomial feature expansion

As previously mentioned, our relationship is of the form:

$$y \approx \hat y = \theta_0 x_p^0 + \theta_1 x_p^1 + \theta_2 x_p^2 + \theta_3 x_p^3 + ... + \theta_d x_p^d = X_{poly} \mathbf{\theta}$$

However, linear regression can only fit a straight line. Thus, if we have sampled datapoints following a non-linear pattern, like curved data or a parabola, we need a polynomial regression model that solves this problem by adding powers of $x$ as new inputs. That way, the model can bend to fit more complex shapes of relationships in the data. For this to work, we first need to transform our inputs $x$ to polynomial feature matrix $X_{poly}$. With this, we transform our problem such that a linear model can actually fit non-linear data. In `scikit-learn` for example, this is done through [`Polynomial Features`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html), however you will implement this manually.

Given an $(n \times 1)$-dimensional input tensor `x` as well as the scalar `degree` $d$, generate a new feature matrix consisting of all polynomial combintions of $x$, aka the matrix $X_{poly}$. The function should return a $(n \times d+1)$-dimensional matrix of type `torch.Tensor`.

In [143]:
def build_polynomial_features(x, degree):
    #TODO: implement polynomial feature expansion
    # YOUR CODE HERE
    raise NotImplementedError

In [48]:
# testing build_polynomial_features
x_test, y_test = generate_datapoints(5, 0.25)
X_poly_test = build_polynomial_features(x_test, 3)

expected_X_poly_test = torch.tensor([[1.0000, 0.8823, 0.7784, 0.6868],
                                [1.0000, 0.9150, 0.8372, 0.7661],
                                [1.0000, 0.3829, 0.1466, 0.0561],
                                [1.0000, 0.9593, 0.9203, 0.8828],
                                [1.0000, 0.3904, 0.1524, 0.0595]])

assert X_poly_test.shape == torch.Size([5, 4])
assert type(X_poly_test) == torch.Tensor
assert torch.allclose(X_poly_test, expected_X_poly_test, atol=1e-3) == True

### Task 2: Fitting the model

Now that you have expanded the input features to their polynomial features, you now want to find the best weight vector $\mathbf{\theta}$, such that multiplying it with our polynomial features comes as close as possible to our true outputs $y$. One of the ways we can solve this problem is the Least Squares method, which minimizes the squared errors between the predictions and real outputs. This is given by:

$$\mathbf{\theta} = (X_{poly}^T X_{poly})^{-1} X_{poly}^T y$$

where $X_{poly}$ is your polynomial feature matrix and $y$ are the true target values. This is also a closed-form solution to linear regression. In this task, you should implement the `fit_least_squares` function, which takes as arguments `X_poly`, your matrix of polynomial features, and the true values `y`. It should implement the above formula and return the optimal $\theta$ vector of dimensionality $(d+1)$. 

In [145]:
def fit_least_squares(X_poly, y):
    #TODO implement least squares
    # YOUR CODE HERE
    raise NotImplementedError

In [53]:
expected_theta_test = torch.tensor([64.6249, -295.1517,  402.4412, -172.2118])
theta_test = fit_least_squares(X_poly_test, y_test)

assert torch.allclose(theta_test, expected_theta_test, atol=1e-3) == True
assert type(theta_test) == torch.Tensor
assert theta_test.shape == torch.Size([4])


### Task 3: Predict outputs

Finally, using a linear approach, we predict our $\hat y$ as:

$$\hat y = X_{poly} \mathbf{\theta}$$

Implement the `predict` function below that does exactly this operation, given `X_poly` as the polynomial feature matrix, and `theta` as the weights $\mathbf{\theta}$, returning $\hat y$ of dimensionality $(n)$

In [147]:
def predict(X_poly, theta):
    # TODO: implement prediction here
    # YOUR CODE HERE
    raise NotImplementedError

In [61]:
y_hat_test = predict(X_poly_test, theta_test)
expected_y_hat_test = torch.tensor([-0.7863, -0.4297,  0.9489, -0.1939,  0.4849])

assert y_hat_test.shape == torch.Size([5])
assert type(y_hat_test) == torch.Tensor
assert torch.allclose(y_hat_test, expected_y_hat_test, atol=1e-3) == True

### Task 4: Evaluating the model

Finally, after you found the optimal weights and made predictions, it is time to measure how good these predictions are. This is what the Mean Squared Error (MSE) is for, which measures how far off your predictions are from the true output. Hence, a smaller MSE is better. We will use the MSE to calculate our empirical risk (error over our training set) and our generalization error (error over our test set).

The MSE is measured as:

$$MSE = \frac{1}{n} \sum^n_{i=1} (y_i - \hat y_i)^2$$

where $n$ is the number of datapoints, $y_i$ is the $i$'th true ouput and $\hat y_i$ is the prediction the model makes for the $i$'th data point. Implement the MSE calculation in the `mean_squared_error` function. The function should return the full mean squared error as `float`.

In [149]:
def mean_squared_error(y_true, y_hat):
    # TODO: Implement MSE formula
    # YOUR CODE HERE
    raise NotImplementedError

In [72]:
mse_test = mean_squared_error(y_test, y_hat_test)

assert type(mse_test) == float
assert torch.allclose(torch.tensor(mse_test), torch.tensor(0.03179203338161631), atol=1e-3)

### Putting it all together

Below we provide a function `plot_polyfit`, which interactively puts your code snippets into a polynomial regression pipeline. It first generates some data with the specified amount of datapoints and noise, then uses `scikit-learn`'s `train_test_split` to split the generated data points into training and test set, builds the polynomial feature matrix out of $x$ and calculates the optimal weights $\theta$. Finally, it predicts the $\hat y$ and computes the training error and test errors. 

In [151]:
def plot_polyfit(n_samples=30, noise=0.1, degree=3):
    # generating data point
    x, y = generate_datapoints(n_samples=n_samples, noise=noise)

    # split data points into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(x.numpy(), y.numpy(), test_size=0.2, random_state=seed)
    X_train = torch.tensor(X_train)
    X_test = torch.tensor(X_test)
    y_train = torch.tensor(y_train)
    y_test = torch.tensor(y_test)

    # building polynomial feature matrix from the X's
    X_train_poly = build_polynomial_features(X_train, degree)
    X_test_poly = build_polynomial_features(X_test, degree)
    
    # compute optimal weights via least squares method
    weights = fit_least_squares(X_train_poly, y_train)
    
    # prepare predictions
    x_plot = torch.linspace(0, 1, steps=100).unsqueeze(1)
    x_plot_poly = build_polynomial_features(x_plot, degree)
    y_plot = predict(x_plot_poly, weights)
    y_train_pred = predict(X_train_poly, weights)
    y_test_pred = predict(X_test_poly, weights)
    
    # compute emp risk and generalization error
    train_error = mean_squared_error(y_train, y_train_pred)
    test_error = mean_squared_error(y_test, y_test_pred)
    
    # plotting
    plt.figure(figsize=(8, 5))
    plt.scatter(X_train, y_train, label="Training data", alpha=0.7)
    plt.scatter(X_test, y_test, label="Test data", color="orange")
    plt.plot(x_plot, y_plot.detach(), 'r-', label=f"Polynomial degree {degree}")
    plt.plot(x_plot, true_polynomial(x_plot.squeeze()).detach().numpy(), label="True function", color="green", linestyle="dashed")
    plt.ylim(-1.5, 1.5)
    plt.legend()
    plt.title(f"Train error: {train_error:.3f} | Test (generalization) error: {test_error:.3f}")
    plt.grid(True)
    plt.show()

Feel free to execute the code below and play around with the sliders in oder to increase/decrease e.g. the number of datapoints as well as the degree of the polynomial that should be predicted to fit the data. You should notice that very low datasamples paired with a higher-degree polynomial will lead to so-called overfitting (try e.g. n_samples=15 and degree=11): While a very high-degree polynomial can fit most/all training set datapoints perfectly, its ability to generalize to the unseen test set points is very bad, leading to a small training error, but a high generalization error.

In [ ]:
interact(plot_polyfit, 
         n_samples=(5, 100, 5),
         noise=(0.0, 0.7, 0.1),
         degree=(0, 15, 1));